In [ ]:
%matplotlib notebook
import sys
sys.path.append("..")
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import json
import argparse
import torch
from torch.optim import lr_scheduler
from tqdm import tqdm
from data_loader import data_loaders as module_data
from model import loss as module_loss
from model import metric as module_metric
from model import model as module_arch
from trainer import Trainer
from utils import Logger
from utils import util
from utils import torchsummary
from utils import viewTraining
from utils import lr_finder
from utils import classActivationMap
import importlib
import math
import torchvision
from torch.nn import functional as F
from torch import topk
import skimage.transform
import jupyter
from IPython import display
print("Modules loaded")

importlib.reload(module_data) #load recent changes to data_loaders.py
importlib.reload(module_arch)
importlib.reload(module_loss)
importlib.reload(module_metric)
importlib.reload(util)
importlib.reload(viewTraining)
importlib.reload(lr_finder)
importlib.reload(classActivationMap)
print("Reload complete")

print("GPUs available: " + str(torch.cuda.device_count()))
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [ ]:
%%javascript
Jupyter.keyboard_manager.disable();

In [ ]:
config_file = '../configs/config_hdf5.json'

# load config file
with open(config_file) as handle:
    config = json.load(handle)

In [ ]:
#This function gets the parameters from the config.json file 
def get_instance(module, name, config, *args):
    return getattr(module, config[name]['type'])(*args, **config[name]['args'])
    
data_loader = get_instance(module_data, 'data_loader', config) #looks in data_loader/data_loaders.py for 'MNISTDataLoader'
print("Total number of training images = " + str(data_loader.dataset.data_len))
data_loader_test = get_instance(module_data, 'data_loader_test', config) #looks in data_loader/data_loaders.py for 'MNISTDataLoader'
print("Total number of testing images = " + str(data_loader_test.dataset.data_len))

normalized = False
util.visualizeBatch(data_loader, normalized) 

In [ ]:
def set_instance(module, name, config, *args):
    setattr(module, config[name]['type'])(*args, **config[name]['args'])
    
# setting path to save trained models and log files
path = os.path.join(config['trainer']['save_dir'], config['name'])

In [ ]:
#train model code

In [ ]:
def main(config, resume):
    train_logger = Logger() #uses entries to store training performance metrics
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # setup data_loader instances
    data_loader = get_instance(module_data, 'data_loader', config) #looks in data_loader/data_loaders.py for 'MNISTDataLoader'
    print("Total number of training images = " + str(data_loader.dataset.data_len))
    valid_data_loader = data_loader.split_validation() #allocate some images as validation

    # build model architecture
    model = get_instance(module_arch, 'arch', config) #looks in the model/model.py for 'MnistModel', as specified by config
    print(model)
    #torchsummary.summary(model, (1,7,32,32))
     
    
    if torch.cuda.is_available():
        print("Using GPU: " + torch.cuda.get_device_name(0))
    else:
        print("Using CPU to train")
    
    # get function handles of loss and metrics
    loss = getattr(module_loss, config['loss']) #looks in model/loss.py for criterion function specified in config
    criterion = loss(data_loader.dataset.weight.to(device)) # for imbalanced datasets
    metrics = [getattr(module_metric, met) for met in config['metrics']] #get all the metrics in model/metrics.py - default is accuracy and top 3 accuracy

    # build optimizer, learning rate scheduler. delete every lines containing lr_scheduler for disabling scheduler
    trainable_params = filter(lambda p: p.requires_grad, model.parameters()) #Number of training params
    optimizer = get_instance(torch.optim, 'optimizer', config, trainable_params)
    lr_scheduler = get_instance(torch.optim.lr_scheduler, 'lr_scheduler', config, optimizer)

    
    trainer = Trainer(model, criterion, metrics, optimizer,
                      resume=resume, #choose a previous epoch to start training from
                      config=config,
                      data_loader=data_loader,
                      valid_data_loader=valid_data_loader,
                      lr_scheduler=lr_scheduler,
                      train_logger=train_logger)
    
    findLR = False
    if findLR:
        #set lr = 1e-7 in config file 
        device_lr = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        lr_finder_training = lr_finder.LRFinder(model, optimizer, criterion, device=device_lr)
        lr_finder_training.range_test(data_loader, end_lr=1, num_iter=100, val_loader = valid_data_loader)
        lr_finder_training.plot(skip_start = 10, skip_end=5)
        # trim the first 10 and last 5 iteration from calculating loss (num_iter > skip_start + skip_end)
    else:
        trainer.train()
        a=1

In [ ]:
# Train the network
main(config, None)

In [ ]:
#test model code

In [ ]:
# build model architecture
model = get_instance(module_arch, 'arch', config)
print(torch.cuda.get_device_name(0))
normalized = False

# get function handles of loss and metrics
loss_fn = getattr(module_loss, config['loss'])
criterion = loss_fn(None)
metric_fns = [getattr(module_metric, met) for met in config['metrics']]

# load state dict
checkpoint = torch.load(resume)
checkpoint_state_dict = checkpoint['state_dict']
for key in list(checkpoint_state_dict.keys()):
    if 'module.' in key:
        checkpoint_state_dict[key.replace('module.', '')] = checkpoint_state_dict[key]
        del checkpoint_state_dict[key]
model.load_state_dict(checkpoint_state_dict)

# prepare model for testing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
def main2(config, resume, data_loader):
    
    model.eval() #tells model to ignore dropout and batch normalization
    total_loss = 0.0
    total_metrics = torch.zeros(len(metric_fns))
    
    classes = ('pt', 'tal')
    all_pred = []
    all_true = []
    all_softmax = []
    
    with torch.no_grad(): #speed up calculations, unable to perform back propogation
        for i, (data, target) in enumerate(tqdm(data_loader)): #tqdm is a progress bar
            data, target = data.to(device), target.to(device)
            output = model(data)
            image = np.squeeze(data[0].cpu().data.numpy())
            label = np.squeeze(target[0].cpu().data.numpy())
            all_true.extend(target.cpu().data.numpy())
            all_pred.extend(np.argmax(output.cpu().data.numpy(), axis=1))
            
            m = torch.nn.Softmax(dim=0)
            for i,row in enumerate(output.cpu()):
                sm = m(row)
                all_softmax.append(sm.data.numpy())
              
            if i < 2:
                m = torch.nn.Softmax(dim=0)
                
            # computing loss, metrics on test set
            loss = criterion(output, target)
            batch_size = data.shape[0]
            total_loss += loss.item() * batch_size
            if output.is_cuda: output = output.cpu()
            if target.is_cuda: target = target.cpu()
            
            for i, metric in enumerate(metric_fns):
                total_metrics[i] += metric(output.cpu(), target) * batch_size
    
    outputcsv = False
    if outputcsv:
        ids = data_loader.dataset.getIds()
        softmax = pd.DataFrame(all_softmax)
        all_true_for_outputcsv = [x+1 for x in all_true]
        frames = [ids, softmax, pd.DataFrame(all_true_for_outputcsv)]
        output_data= np.concatenate(frames, axis=1)
        print(output_data.shape)
        output_df = pd.DataFrame(output_data)
        output_df.to_csv('overlaycsv.csv', index=False,  header=False)
        
    n_samples = len(data_loader.sampler)
    print("num test images = " + str(n_samples))
    log = {'loss': total_loss / n_samples}
    log.update({met.__name__: total_metrics[i].item() / n_samples for i, met in enumerate(metric_fns)})
    for key in log:
        print("{} = {:.4f}".format(key, log[key]))
    log['classes'] = classes
    log['test_targets'] = all_true
    log['test_predictions'] = all_pred
    print("My_metric is accuracy")
    util.plot_confusion_matrix(all_true, all_pred, classes=classes, normalize=False)

In [ ]:
# setup data_loader instances
data_loader = get_instance(module_data, 'data_loader_test', config)
#Test
main2(config, resume, data_loader)